In [13]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel

In [11]:
warnings.filterwarnings('ignore')

In [2]:
# 设置字符集，防止图片中的中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# 设置jupyter图片显示方式
%matplotlib tk

In [4]:
# 特征之后的数据读取
data = pd.read_csv("../data/features01.csv")
data.head(3)

,loan_amnt,int_rate,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,debt_settlement_flag_N,debt_settlement_flag_Y
0,5000.0,10.65,24000.0,1.0,27.65,0.0,1.0,0.0,3.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,30000.0,0.0,1.00,0.0,5.0,0.0,3.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,2400.0,15.96,12252.0,1.0,8.72,0.0,2.0,0.0,2.0,0.0,...,0,0,0,0,0,1,0,0,1,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39786 entries, 0 to 39785
Data columns (total 56 columns):
loan_amnt                              39786 non-null float64
int_rate                               39786 non-null float64
annual_inc                             39786 non-null float64
loan_status                            39786 non-null float64
dti                                    39786 non-null float64
delinq_2yrs                            39786 non-null float64
inq_last_6mths                         39786 non-null float64
mths_since_last_delinq                 39786 non-null float64
open_acc                               39786 non-null float64
pub_rec                                39786 non-null float64
revol_bal                              39786 non-null float64
revol_util                             39786 non-null float64
total_acc                              39786 non-null float64
collections_12_mths_ex_med             39786 non-null float64
acc_now_delinq         

In [6]:
# 获取X和Y
Y = data['loan_status']
X = data.drop(['loan_status'], 1, inplace=False)
print("样本数量为:%d, 特征属性数量为:%d" % X.shape)

样本数量为:39786, 特征属性数量为:55


In [7]:
# 样本的分割
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("训练集合数据量:%d,%d" % x_train.shape)
print("测试集合数据量:%d,%d" % x_test.shape)

训练集合数据量:31828,55
测试集合数据量:7958,55


In [8]:
# 一般情况下：在做分类的时候，都会看一下各个类别的样本数量的比例，看一下是否存在数据的不平衡情况
print(y_train.value_counts())
print(y_test.value_counts())

1.0    27301
0.0     4527
Name: loan_status, dtype: int64
1.0    6815
0.0    1143
Name: loan_status, dtype: int64


In [12]:
# 首先做一个最优参数的构造
parameters = {
    "penalty": ['l1', 'l2'],
    "C": [0.01, 0.1, 1],
    "fit_intercept": [True, False],
    "max_iter": [100, 150, 200]
}
clf = GridSearchCV(LogisticRegression(random_state=0), param_grid=parameters, cv=3)
clf.fit(x_train, y_train)

# 得到最优参数
print("最优参数:", end="")
print(clf.best_params_)

最优参数:{'C': 0.01, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1'}


In [20]:
# 使用逻辑回归来分析数据
lr = LogisticRegression(C = 0.1, fit_intercept=True, max_iter=100, penalty='l1', random_state=0)
lr.fit(x_train, y_train)

train_predict = lr.predict(x_train)
test_predict = lr.predict(x_test)

In [21]:
print("训练数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_train, train_predict)))
print("训练数据分类报告:\n{}\n".format(metrics.classification_report(y_train, train_predict)))
print("测试数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_test, test_predict)))
print("测试数据分类报告:\n{}\n".format(metrics.classification_report(y_test, test_predict)))

训练数据混淆矩阵:
[[  111  4416]
 [   57 27244]]

训练数据分类报告:
              precision    recall  f1-score   support

         0.0       0.66      0.02      0.05      4527
         1.0       0.86      1.00      0.92     27301

   micro avg       0.86      0.86      0.86     31828
   macro avg       0.76      0.51      0.49     31828
weighted avg       0.83      0.86      0.80     31828


测试数据混淆矩阵:
[[  32 1111]
 [  12 6803]]

测试数据分类报告:
              precision    recall  f1-score   support

         0.0       0.73      0.03      0.05      1143
         1.0       0.86      1.00      0.92      6815

   micro avg       0.86      0.86      0.86      7958
   macro avg       0.79      0.51      0.49      7958
weighted avg       0.84      0.86      0.80      7958




In [22]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.9241
测试集合上的f1指标:0.9238
训练集合上的召回率(y=0):0.0245
训练集合上的召回率(y=0):0.0280
训练集合上的召回率(y=1):0.9979
训练集合上的召回率(y=1):0.9982


In [23]:
# 使用逻辑回归来分析数据 + 可以选择给类别添加权重
weight = {
    0: 3, # 在模型训练和测试的过程中，类别0的重要性
    1: 1 # 在模型训练和测试的过程中，类别1的重要性
}
lr = LogisticRegression(C = 0.1, 
                        fit_intercept=True, 
                        max_iter=100, 
                        penalty='l1', 
                        random_state=0,
                        class_weight=weight
                       )
lr.fit(x_train, y_train)

train_predict = lr.predict(x_train)
test_predict = lr.predict(x_test)

In [24]:
print("训练数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_train, train_predict)))
print("训练数据分类报告:\n{}\n".format(metrics.classification_report(y_train, train_predict)))
print("测试数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_test, test_predict)))
print("测试数据分类报告:\n{}\n".format(metrics.classification_report(y_test, test_predict)))

训练数据混淆矩阵:
[[ 1258  3269]
 [ 2587 24714]]

训练数据分类报告:
              precision    recall  f1-score   support

         0.0       0.33      0.28      0.30      4527
         1.0       0.88      0.91      0.89     27301

   micro avg       0.82      0.82      0.82     31828
   macro avg       0.61      0.59      0.60     31828
weighted avg       0.80      0.82      0.81     31828


测试数据混淆矩阵:
[[ 303  840]
 [ 644 6171]]

测试数据分类报告:
              precision    recall  f1-score   support

         0.0       0.32      0.27      0.29      1143
         1.0       0.88      0.91      0.89      6815

   micro avg       0.81      0.81      0.81      7958
   macro avg       0.60      0.59      0.59      7958
weighted avg       0.80      0.81      0.81      7958




In [25]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.8941
测试集合上的f1指标:0.8927
训练集合上的召回率(y=0):0.2779
训练集合上的召回率(y=0):0.2651
训练集合上的召回率(y=1):0.9052
训练集合上的召回率(y=1):0.9055


In [26]:
# 使用随机森林来分析数据
forest = RandomForestClassifier(random_state=0, max_depth=5)
forest.fit(x_train, y_train)

train_predict = forest.predict(x_train)
test_predict = forest.predict(x_test)

In [27]:
print("训练数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_train, train_predict)))
print("训练数据分类报告:\n{}\n".format(metrics.classification_report(y_train, train_predict)))
print("测试数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_test, test_predict)))
print("测试数据分类报告:\n{}\n".format(metrics.classification_report(y_test, test_predict)))

训练数据混淆矩阵:
[[   68  4459]
 [    2 27299]]

训练数据分类报告:
              precision    recall  f1-score   support

         0.0       0.97      0.02      0.03      4527
         1.0       0.86      1.00      0.92     27301

   micro avg       0.86      0.86      0.86     31828
   macro avg       0.92      0.51      0.48     31828
weighted avg       0.88      0.86      0.80     31828


测试数据混淆矩阵:
[[  15 1128]
 [   3 6812]]

测试数据分类报告:
              precision    recall  f1-score   support

         0.0       0.83      0.01      0.03      1143
         1.0       0.86      1.00      0.92      6815

   micro avg       0.86      0.86      0.86      7958
   macro avg       0.85      0.51      0.47      7958
weighted avg       0.85      0.86      0.79      7958




In [28]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.9245
测试集合上的f1指标:0.9233
训练集合上的召回率(y=0):0.0150
训练集合上的召回率(y=0):0.0131
训练集合上的召回率(y=1):0.9999
训练集合上的召回率(y=1):0.9996


In [29]:
# 使用随机森林来分析数据 + 权重
weight = {
    0: 3, # 在模型训练和测试的过程中，类别0的重要性
    1: 1 # 在模型训练和测试的过程中，类别1的重要性
}
forest = RandomForestClassifier(random_state=0, max_depth=5, class_weight=weight)
forest.fit(x_train, y_train)

train_predict = forest.predict(x_train)
test_predict = forest.predict(x_test)

In [30]:
print("训练数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_train, train_predict)))
print("训练数据分类报告:\n{}\n".format(metrics.classification_report(y_train, train_predict)))
print("测试数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_test, test_predict)))
print("测试数据分类报告:\n{}\n".format(metrics.classification_report(y_test, test_predict)))

训练数据混淆矩阵:
[[  208  4319]
 [  170 27131]]

训练数据分类报告:
              precision    recall  f1-score   support

         0.0       0.55      0.05      0.08      4527
         1.0       0.86      0.99      0.92     27301

   micro avg       0.86      0.86      0.86     31828
   macro avg       0.71      0.52      0.50     31828
weighted avg       0.82      0.86      0.80     31828


测试数据混淆矩阵:
[[  43 1100]
 [  56 6759]]

测试数据分类报告:
              precision    recall  f1-score   support

         0.0       0.43      0.04      0.07      1143
         1.0       0.86      0.99      0.92      6815

   micro avg       0.85      0.85      0.85      7958
   macro avg       0.65      0.51      0.50      7958
weighted avg       0.80      0.85      0.80      7958




In [31]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.9236
测试集合上的f1指标:0.9212
训练集合上的召回率(y=0):0.0459
训练集合上的召回率(y=0):0.0376
训练集合上的召回率(y=1):0.9938
训练集合上的召回率(y=1):0.9918


In [32]:
# 基于随机森林获取影响放贷的二十大因素
feature_importances = forest.feature_importances_
feature_importances = 100.0 * (feature_importances / feature_importances.max())

indices = np.argsort(feature_importances)[-20:]
plt.barh(np.arange(20), feature_importances[indices], color='dodgerblue', alpha=0.4)
plt.yticks(np.arange(20 + 0.25), np.array(X.columns)[indices])
plt.xlabel('特征重要性百分比')
plt.title('随机森林20大重要特征提取')
plt.show()

In [33]:
print("权重大于0的特征数目:{}".format(np.sum(forest.feature_importances_ > 0)))
print("权重等于0的特征数目:{}".format(np.sum(forest.feature_importances_ == 0)))
print("权重小于0的特征数目:{}".format(np.sum(forest.feature_importances_ < 0)))

权重大于0的特征数目:41
权重等于0的特征数目:14
权重小于0的特征数目:0


In [40]:
# 用随机森林选择特征，然后使用Logistic回归来做预测
weight = {
    0: 3, # 在模型训练和测试的过程中，类别0的重要性
    1: 1 # 在模型训练和测试的过程中，类别1的重要性
}
# a. 特征选择过程
print("原始样本大小:{}".format(x_train.shape))
forest = RandomForestClassifier(random_state=0, max_depth=5, class_weight=weight)
# 当特征的权重大于等于给定的threshold的时候，该特征就保留；由于随机森林中的特征属性权重一定是大于等于0的值，所以一般情况下，，
# 在决策树类型的算法中，使用SelectFromModel一般选择比0稍大一点点的阈值。
sm = SelectFromModel(estimator=forest, threshold=0.0000001)
sm.fit(x_train, y_train)
x_train1 = sm.transform(x_train)
x_test1 = sm.transform(x_test)
print("原始样本大小:{}".format(x_train1.shape))
# b. logistic回归训练
lr = LogisticRegression(C = 0.1, class_weight=weight, fit_intercept=True, max_iter=100, penalty='l1', random_state=0)
lr.fit(x_train1, y_train)

train_predict = lr.predict(x_train1)
test_predict = lr.predict(x_test1)

原始样本大小:(31828, 55)
原始样本大小:(31828, 41)


In [41]:
print("训练数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_train, train_predict)))
print("训练数据分类报告:\n{}\n".format(metrics.classification_report(y_train, train_predict)))
print("测试数据混淆矩阵:\n{}\n".format(metrics.confusion_matrix(y_test, test_predict)))
print("测试数据分类报告:\n{}\n".format(metrics.classification_report(y_test, test_predict)))

训练数据混淆矩阵:
[[ 1255  3272]
 [ 2594 24707]]

训练数据分类报告:
              precision    recall  f1-score   support

         0.0       0.33      0.28      0.30      4527
         1.0       0.88      0.90      0.89     27301

   micro avg       0.82      0.82      0.82     31828
   macro avg       0.60      0.59      0.60     31828
weighted avg       0.80      0.82      0.81     31828


测试数据混淆矩阵:
[[ 299  844]
 [ 651 6164]]

测试数据分类报告:
              precision    recall  f1-score   support

         0.0       0.31      0.26      0.29      1143
         1.0       0.88      0.90      0.89      6815

   micro avg       0.81      0.81      0.81      7958
   macro avg       0.60      0.58      0.59      7958
weighted avg       0.80      0.81      0.80      7958




In [39]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.8953
测试集合上的f1指标:0.8909
训练集合上的召回率(y=0):0.2691
训练集合上的召回率(y=0):0.2450
训练集合上的召回率(y=1):0.9087
训练集合上的召回率(y=1):0.9051


In [42]:
# GBDT的提取的效果
gbdt = GradientBoostingClassifier(min_samples_split=50, max_depth=2, n_estimators=300, learning_rate=0.1, random_state=0)
gbdt.fit(x_train, y_train)

train_predict = gbdt.predict(x_train)
test_predict = gbdt.predict(x_test)

In [43]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.9249
测试集合上的f1指标:0.9228
训练集合上的召回率(y=0):0.0320
训练集合上的召回率(y=0):0.0262
训练集合上的召回率(y=1):0.9984
训练集合上的召回率(y=1):0.9966


In [44]:
# 基于GBDT获取影响放贷的二十大因素
feature_importances = gbdt.feature_importances_
feature_importances = 100.0 * (feature_importances / feature_importances.max())

indices = np.argsort(feature_importances)[-20:]
plt.barh(np.arange(20), feature_importances[indices], color='dodgerblue', alpha=0.4)
plt.yticks(np.arange(20 + 0.25), np.array(X.columns)[indices])
plt.xlabel('特征重要性百分比')
plt.title('GBDT 20大重要特征提取')
plt.show()

In [45]:
print("权重大于0的特征数目:{}".format(np.sum(gbdt.feature_importances_ > 0)))
print("权重等于0的特征数目:{}".format(np.sum(gbdt.feature_importances_ == 0)))
print("权重小于0的特征数目:{}".format(np.sum(gbdt.feature_importances_ < 0)))

权重大于0的特征数目:38
权重等于0的特征数目:17
权重小于0的特征数目:0


In [19]:
# TODO: 实现以下使用GBDT提取特征(维度扩展)，提取出来的特征使用Logistic回归做一个分类，看一下模型效果如何


In [46]:
# 在实际工作中，如果发现模型的效果不如意，那么可能需要考虑特征选择和降维
# 使用逻辑回归来分析数据 + 特征选择 + 降维
# 特征选择：从所有特征属性中抽取出来影响目标属性(target)效果最大的特征属性作为下一步的特征属性列表\
# 很多特征选择工程都是选择方差比较大特征属性
# 也可以使用随机森林、GBDT、决策树来进行特征选择

# 降维：压缩样本的维度空间，直白来讲，就是讲DataFrame中原本的多个列合并成为一列

# 1. 特征选择
feature_importances = gbdt.feature_importances_
indices = np.argsort(feature_importances)[-30:]
top30_features = np.array(X.columns)[indices]

# 2. 提取影响最大的三十个特征属性
x_train2 = x_train[top30_features]
x_test2 = x_test[top30_features]

# 3. 降维处理(在三十个特征之外，将其它的特征数据做一个降维操作)
x_train3 = x_train.drop(top30_features, 1, inplace=False)
x_test3 = x_test.drop(top30_features, 1, inplace=False)
pca = PCA(n_components=10)
pca.fit(x_train3)
x_test3 = pca.transform(x_test3)
x_train3 = pca.transform(x_train3)

# 4. 两个DataFrame合并
x_train2 = np.hstack([x_train2, x_train3])
x_test2 = np.hstack([x_test2, x_test3])
print("合并后样本大小:{}".format(x_train2.shape))

weight = {
    0: 3, # 在模型训练和测试的过程中，类别0的重要性
    1: 1 # 在模型训练和测试的过程中，类别1的重要性
}
lr = LogisticRegression(C = 0.1,class_weight=weight, fit_intercept=True, max_iter=100, penalty='l1', random_state=0)
lr.fit(x_train2, y_train)

train_predict = lr.predict(x_train2)
test_predict = lr.predict(x_test2)

合并后样本大小:(31828, 40)


In [47]:
print("训练集合上的f1指标:%.4f" % f1_score(y_train, train_predict))
print("测试集合上的f1指标:%.4f" % f1_score(y_test, test_predict))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_train, train_predict, pos_label=0))
print("训练集合上的召回率(y=0):%.4f" % recall_score(y_test, test_predict, pos_label=0))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_train, train_predict, pos_label=1))
print("训练集合上的召回率(y=1):%.4f" % recall_score(y_test, test_predict, pos_label=1))

训练集合上的f1指标:0.8940
测试集合上的f1指标:0.8926
训练集合上的召回率(y=0):0.2770
训练集合上的召回率(y=0):0.2686
训练集合上的召回率(y=1):0.9052
训练集合上的召回率(y=1):0.9049


### 计算KS指标

In [50]:
# 计算KS的方式一：
def compute_ks(data):
    # 按照样本为正样本的概率值升序排序 ，也即坏样本的概率从高到低排序
    sorted_list = data.sort_values(['predict_proba'], ascending=[True])
    total_good = sorted_list['label'].sum()
    total_bad = sorted_list.shape[0] - total_good  
    
    max_ks = 0.0
    good_count = 0.0
    bad_count = 0.0
    bads = []
    goods = []
    for index, row in sorted_list.iterrows(): #按照标签和每行拆开
        if row['label'] == 0:
            bad_count +=1
        else:
            good_count +=1
        val = abs(bad_count/total_bad - good_count/total_good)
        if index % 100 == 0:
            bads.append(bad_count/total_bad)
            goods.append(good_count/total_good)
        max_ks = max(max_ks, val)
    # 可视化画图
    plt.plot(range(len(bads)), bads, 'r-o')
    plt.plot(range(len(goods)), goods, 'g-o')
    plt.show()
    return max_ks

test_pd=pd.DataFrame()
y_predict_proba=lr.predict_proba(x_test2)[:,1] #取被分为正样本的概率那一列
Y_test_1=np.array(y_test)
test_pd['label']=Y_test_1
test_pd['predict_proba']=y_predict_proba
print ("测试集 KS:",compute_ks(test_pd))

测试集 KS: 0.27886314284082064


In [52]:
# 计算KS的方式二
y_predict_proba = lr.predict_proba(x_test2)[:,1]
fpr,tpr,thresholds= sklearn.metrics.roc_curve(np.array(y_test),y_predict_proba)
print ('KS:',max(tpr-fpr))

KS: 0.2788631428408206
